In [82]:
import pandas as pd
import numpy as np
import json
import networkx as nx
import time
import heapq
import subprocess
from subprocess import Popen, PIPE, STDOUT
import time
!chmod +x spike-dasm.exe

In [83]:
# Read JSON
with open('rv64um-v-mul.out', 'r') as f:
    json_lines = f.readlines()

inst_jsons = []
for line in json_lines:
    try:
        inst_jsons.append(json.loads(line))
    except json.JSONDecodeError:
        pass

In [84]:
p = Popen("./spike-dasm.exe --isa=rv64gcv", stdout=PIPE, stdin=PIPE, stderr=PIPE, text=True, shell=True)
stdout_data = p.communicate(input=f"DASM(0x0001041f)")
stdout_data

('unknown\n', '')

In [85]:
def generate_data_array(jsons):
    arr = []
    for json in jsons:
        if json["event_name"] == "EX":
            p = Popen("./spike-dasm.exe --isa=rv64gcv", stdout=PIPE, stdin=PIPE, stderr=PIPE, text=True, shell=True)
            stdout_data = p.communicate(input=f"DASM({json['data']})")[0].strip()
            arr.append(stdout_data)
        else:
            arr.append(json["data"])
    return np.array(arr)
inst_ids = np.array([inst_jsons[i]["id"] for i in range(len(inst_jsons))])
inst_cycle = np.array([inst_jsons[i]["cycle"].strip() for i in range(len(inst_jsons))])
inst_event = np.array([inst_jsons[i]["event_name"] for i in range(len(inst_jsons))])
data_field = generate_data_array(inst_jsons)
inst_parent = np.array([inst_jsons[i]["parents"] for i in range(len(inst_jsons))])
data = np.column_stack((inst_ids,inst_parent, inst_cycle, inst_event, data_field))
columns = ["inst_id", "parent_id", "cycle", "stage", "data"]
df = pd.DataFrame(data=data, columns=columns)

In [86]:
df[df["data"].str.contains("mul")]

,inst_id,parent_id,cycle,stage,data
7189,0x0000000400083e6c,0x0000000100083e6a,540268,EX,"mul a1, a1, a5"
7210,0x0000000400083e7c,0x0000000100083e6f,540284,EX,"mul a1, a1, a5"
25269,0x0000000400085dcb,0x0000000100085dca,548299,EX,"mul a4, ra, sp"
25421,0x0000000400085df9,0x0000000100085df8,548345,EX,"mul a4, ra, sp"
25527,0x0000000400085e22,0x0000000100085e21,548386,EX,"mul a4, ra, sp"
25567,0x0000000400085e2b,0x0000000100085e27,548395,EX,"mul a4, ra, sp"
25607,0x0000000400085e34,0x0000000100085e30,548404,EX,"mul a4, ra, sp"
25705,0x0000000400085e59,0x0000000100085e58,548441,EX,"mul a4, ra, sp"
25745,0x0000000400085e62,0x0000000100085e5e,548450,EX,"mul a4, ra, sp"
25850,0x0000000400085e8f,0x0000000100085e8e,548495,EX,"mul a4, ra, sp"


In [99]:
def construct_graph(df):
    DG = nx.DiGraph()
    for row in df.itertuples():
        DG.add_node(row.inst_id, cycle=row.cycle, data=row.data, stage=row.stage)
        if row.parent_id != "None":
            DG.add_edge(row.parent_id, row.inst_id)
    return DG            

def construct_speculative_trace(G):
    paths = []
    id = 0
    for node in G:
        data = G.nodes[node]
        if G.in_degree(node) == 0: # root node
            new_paths = trace_down(G, node, [], [])
            paths.extend(new_paths)
    for path in paths:
        if path[-1][0] != "RET":
            path.append(("FLUSH", str(int(path[-1][1]) + 1), None))
        path.insert(0, (id, path[0][-1]))
    return paths

def trace_down(G, node, curr_path, paths):
    data = G.nodes[node]
    curr_path.append((data["stage"], data["cycle"], data["data"]))
    if G.out_degree(node) == 0: # terminal node
        paths.append(curr_path)
        return paths
    succs = list(DG.successors(node))
    if data["stage"] == "EX" and len(succs) > 1:
        inst_paths = [trace_down(G, n, [], [])[0] for n in succs]
        inst_paths = inst_paths[0] + inst_paths[1]
        inst_paths.sort(key=lambda x: x[1])
        merged_path = []
        for i, p in enumerate(inst_paths):
            if p[0] == "WB":
                continue
            elif p[0] == "RET":
                continue
            elif p[0] == "DIV-WB":
                merged_path.append(("WB", p[1], p[2]))
                merged_path.append(("RET", str(int(p[1]) + 1), p[2]))
            else:
                merged_path.append(p)
        print(merged_path)
        paths.append(curr_path + merged_path)
    else:
        for n in succs:
            paths.extend(trace_down(G, n, curr_path[:], []))
    return paths

def construct_committed_trace(G):
    paths = []
    id = 0
    for node in G:
        data = G.nodes[node]
        if G.out_degree(node) == 0 and data["stage"] == "RET": # committed leaf node
            new_path = trace_up(G, node)
            new_path.insert(0, (id, data["data"]))
            paths.append(new_path)
            id += 1
    return paths

def trace_up(G, node):
    path = []
    while node:
        data = G.nodes[node]
        path.insert(0, (data["stage"], data["cycle"], data["data"]))
        node = list(DG.predecessors(node))[0] if list(DG.predecessors(node)) else ""
    return path
    

In [94]:
DG = construct_graph(df)

In [7]:
paths = construct_committed_trace(DG)

In [100]:
spec_paths = construct_speculative_trace(DG)

[('MEM', '540269', '0x00800020a8'), ('DIV', '540269', '0x0')]
[('MEM', '540285', '0x00800020a8'), ('DIV', '540285', '0x0'), ('WB', '540288', '0x0'), ('RET', '540289', '0x0')]
[('MEM', '548300', '0x00000029b4'), ('DIV', '548300', '0x0'), ('WB', '548305', '0x0'), ('RET', '548306', '0x0')]
[('MEM', '548346', '0x00000029f0'), ('DIV', '548346', '0x0'), ('WB', '548351', '0x0'), ('RET', '548352', '0x0')]
[('MEM', '548387', '0x0000002a0c'), ('DIV', '548387', '0x0'), ('WB', '548391', '0x0'), ('RET', '548392', '0x0')]
[('MEM', '548396', '0x0000002a24'), ('DIV', '548396', '0x0'), ('WB', '548400', '0x0'), ('RET', '548401', '0x0')]
[('MEM', '548405', '0x0000002a3c'), ('DIV', '548405', '0x0'), ('WB', '548408', '0x0'), ('RET', '548409', '0x0')]
[('MEM', '548442', '0x0000002a54'), ('DIV', '548442', '0x0'), ('WB', '548446', '0x0'), ('RET', '548447', '0x0')]
[('MEM', '548451', '0x0000002a6c'), ('DIV', '548451', '0x0'), ('WB', '548460', '0x0'), ('RET', '548461', '0x0')]
[('MEM', '548496', '0x0000002a84')

In [96]:
filtered_paths = [p for p in spec_paths if int(p[0][1], 0) >= 0x80000000] 

In [97]:
import heapq

def convert_to_kanata(threads, verbose=False):
    pq = []
    id = 0
    if not verbose:
        threads = list(filter(lambda x: x[-1][0] == 'RET', threads)) #Relies on the last element of inst list being RET
    for inst in threads:
        for stage in inst[1:]:
            heapq.heappush(pq, ((int(stage[1])), (id, stage[2], stage[0]))) #Min heap of (cycle -> (unique_id, pc, pipeline stage))
        id += 1
            
    with open('mulf.log', 'w') as file:
        file.write('Kanata    0004\n')
        cycle, (id, pc, stage) = heapq.heappop(pq)
        prev_cycle = cycle
        file.write(f'C=\t{cycle}\n')
        while pq:
            cycle_diff = cycle - prev_cycle
            if (cycle_diff > 0):
                file.write(f"C\t{cycle_diff}\n")
            if (stage == 'IF1'):
                file.write(f"I\t{id}\t{cycle}\t0\n")
                # file.write(f"L    {id}    0    {pc}\n")
            if (stage == 'RET'):
                file.write(f"R\t{id}\t{id}\t0\n")
            elif (stage == 'FLUSH'):
                file.write(f"R\t{id}\t{id}\t1\n")
            elif (stage == "MEM"):
                file.write(f"S\t{id}\t0\t{stage}\n")
                file.write(f"L\t{id}\t0\tPC:{pc}\n")
            elif (stage == "EX"):
                file.write(f"S\t{id}\t0\t{stage}\n")
                file.write(f"L\t{id}\t0\t{pc} \n")
            else:
                file.write(f"S\t{id}\t0\t{stage}\n")
                file.write(f"L\t{id}\t1\tPC:{pc}\n")

            prev_cycle = cycle
            cycle, (id, pc, stage) = heapq.heappop(pq)

In [98]:
convert_to_kanata(filtered_paths, verbose=True)